In [117]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import *

from qiskit_aer import Aer

We can simulate without using `qiskit` just by using python packages. There we don't need to worry about the initial state $\rho$. We assume it is given to us, in classical matrix form. So the expectation values can be calculated just by calculating traces.

# Notes
Let, $\rho = \sum_{i=1}^d c_i X_i$

It is assumed that $X_i$'s are independent and orthonormal i.e $Tr(X_i^\dagger X_j) = \delta_{ij}$.

Then, $Tr(\rho X_i) = c_i$

Our initial state is $\rho = \rho_1 \oplus \rho_2 \oplus \rho_3$, where,

$$\rho_1 = \frac{1}{3}
\begin{bmatrix}
    1 & 1 & 1 \\
    1 & 1 & 1 \\
    1 & 1 & 1
\end{bmatrix}
            \quad 
\rho_2 = \frac{1}{3}
\begin{bmatrix}
    1 & 1 & 1 \\
    1 & 1 & -1 \\
    1 & -1 & 1
\end{bmatrix} \quad
\rho_3 = \frac{1}{2}
\begin{bmatrix}
    1 & 0 \\ 0 & 1
\end{bmatrix}$$


We want to find `Pauli Decomposition` of $\rho$ i.e. to find $c_i$'s where $X_i$'s are Pauli matrices/tensor products of Pauli matrices.

# Reference from [here](https://docs.quantum.ibm.com/build/specify-observables-pauli)

In [118]:
#TODO: initialize a mixed state
from scipy.linalg import block_diag
from qiskit.quantum_info import DensityMatrix


# specify the quantum state in an array
rho_1 = (1/3)*np.array([[1,1,1],
                        [1,1,1],
                        [1,1,1]])

rho_2 = (1/3)*np.array([[1,1,1],
                        [1,1,-1],
                        [1,-1,1]])

rho_3 = (1/2)*np.array([[1,0],
                        [0,1]])

# direct sum of rho_1, rho_2, and rho_3

rho =  block_diag(rho_1, rho_2, rho_3)
# this is our initial mixed state

#convert rho into density matrix
rho = DensityMatrix(rho)

In [119]:
DensityMatrix.is_valid(rho)

False

In [120]:
# Pauli decomposition
from qiskit.quantum_info import SparsePauliOp

pauli_decomposition = SparsePauliOp.from_operator(rho)

In [121]:
# pauli decomposition
paulis = pauli_decomposition.to_list()

paulis

[('III', (0.375+0j)),
 ('IXI', (0.08333333333333333+0j)),
 ('IXX', (0.08333333333333333+0j)),
 ('IXZ', (0.08333333333333333+0j)),
 ('IYY', (0.08333333333333333-0j)),
 ('IZI', (-0.04166666666666666+0j)),
 ('XXI', (0.08333333333333333+0j)),
 ('XXX', (0.08333333333333333+0j)),
 ('XXZ', (-0.08333333333333333+0j)),
 ('XYY', (-0.08333333333333333+0j)),
 ('YXY', (0.08333333333333333-0j)),
 ('YYI', (0.08333333333333333-0j)),
 ('YYX', (0.08333333333333333-0j)),
 ('YYZ', (-0.08333333333333333+0j)),
 ('ZII', (-0.04166666666666667+0j)),
 ('ZIX', (0.16666666666666666+0j)),
 ('ZXI', (0.08333333333333333+0j)),
 ('ZXX', (0.08333333333333333+0j)),
 ('ZXZ', (0.08333333333333333+0j)),
 ('ZYY', (0.08333333333333333-0j)),
 ('ZZI', (0.04166666666666667+0j)),
 ('ZZX', (0.16666666666666666+0j))]

## Here though we have decomposed our state $\rho$ into Pauli matrices, we don't know how to initialize our circuit in this particular state 🥲

# Let us also try implementing this _from scratch_ without using `qiskit`.

In [122]:
import numpy as np
import sympy as sp

In [123]:
import numpy as np

# check if a given matrix is a density matrix or not
def is_density_matrix(matrix):
    # Check if the matrix is square
    if matrix.shape[0] != matrix.shape[1]:
        print("Matrix is not square")
        return False
    
    # Check if the matrix is Hermitian
    if not np.allclose(matrix, matrix.conj().T):
        print("Matrix is not Hermitian")
        return False
    
    # Check if the trace of the matrix is 1
    if not np.isclose(np.trace(matrix), 1):
        print("Trace of the matrix is not 1")
        return False
    
    # Check if all eigenvalues are non-negative
    eigenvalues = np.linalg.eigvalsh(matrix)
    if not np.all(eigenvalues >= 0):
        print("Not all eigenvalues are non-negative")
        return False
    
    return True

In [128]:
# # generating a random density matrix from scratch

# # Generate a random Hermitian matrix
# random_matrix = np.random.rand(2, 2) + 1j * np.random.rand(2, 2)
# hermitian_matrix = (random_matrix + random_matrix.conj().T) / 2

# # Normalize the matrix to have trace 1
# trace = np.trace(hermitian_matrix)
# density_matrix = hermitian_matrix / trace

# print(density_matrix)   
# is_density_matrix(density_matrix)

# # not robust. prone to falure because the generated matrix may not be positive semi-definite.
# commenting it. Not useful

In [125]:
# generating density matrix using qutip
from qutip import *
dm = rand_dm(5)
print(dm)
is_density_matrix(dm.full())
  

Quantum object: dims=[[5], [5]], shape=(5, 5), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.21814351+1.80507999e-19j -0.03409614-1.41376731e-02j
   0.06507789+4.15756058e-02j -0.07890715+2.80457962e-02j
  -0.06941368+1.77976479e-02j]
 [-0.03409614+1.41376731e-02j  0.1401136 +1.15940308e-19j
   0.02250301+4.06401681e-02j  0.00345036-3.12779565e-02j
   0.04094934-1.41055917e-01j]
 [ 0.06507789-4.15756058e-02j  0.02250301-4.06401681e-02j
   0.1075412 +8.89875101e-20j -0.01063591-3.05772044e-02j
  -0.04286002-3.12151919e-02j]
 [-0.07890715-2.80457962e-02j  0.00345036+3.12779565e-02j
  -0.01063591+3.05772044e-02j  0.26193174+2.16741605e-19j
  -0.0446584 -6.66845198e-02j]
 [-0.06941368-1.77976479e-02j  0.04094934+1.41055917e-01j
  -0.04286002+3.12151919e-02j -0.0446584 +6.66845198e-02j
   0.27226996-6.02177423e-19j]]


True

In [126]:
I, X, Y, Z = qeye(2), sigmax(), sigmay(), sigmaz()
I, X, Y, Z = np.array(I), np.array(X), np.array(Y), np.array(Z)